In [1]:
#Bestehende Daten EInlesen
import numpy
from sklearn.metrics import accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, \
    grabocka_params_to_shapelet_size_dict
from tslearn.utils import ts_size
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
import sktime
from tslearn.preprocessing import TimeSeriesResampler
import random
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test

In [2]:
df = pd.read_pickle("Train.pkl")
df.drop("Unnamed: 0",axis=1,inplace=True)
df.drop(df.loc[(df.Batch == 238)&(df["CuStepNo ValueY"] == 3)].tail(1).index,inplace=True)

In [3]:
#Szenario es kommen neue Daten an (beispielsweise ein BAtch mit 450 Zeilen)
df_test = pd.read_csv("Batch_76.csv")
df_test.drop("Unnamed: 0",axis=1,inplace=True)
df_test1 = df_test[:450]

In [4]:
#Dann mit den neuen Daten unsere alten Daten trainieren und nen score für den Teil des Batches abgeben
df1 = df.loc[df["CuStepNo ValueY"].isin(df_test1["CuStepNo ValueY"].unique())]
for i in df1["Batch"].unique():
    df1.drop(df1.loc[(df1["Batch"] == i)&(df1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1])][df_test1[df_test1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1]].__len__():].index,inplace=True)

C:\Users\paulh\AppData\Local\Temp\ipykernel_8516\3214082494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(df1.loc[(df1["Batch"] == i)&(df1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1])][df_test1[df_test1["CuStepNo ValueY"] == df_test1["CuStepNo ValueY"].unique()[-1]].__len__():].index,inplace=True)


In [5]:
#Trainingsdatensatz machen
X_train1 = df1.drop(["Next_Step","timestamp"],axis=1)
y_train1 = []
X_train_step1 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)].drop(["Batch","CuStepNo ValueY","DeviationID ValueY"],axis=1))
        y_train1.append(X_train1.loc[(X_train1["Batch"] == i)]["DeviationID ValueY"].unique()[0])
        X_train_step1.append(num)

In [6]:
#Länge für die Interpolation bestimmen
length = []
for i in X_train_step1:
    length.append(i.__len__())
X_train_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step1)
#Reshapen
X_train = X_train_s1.reshape(X_train_s1.shape[0],39,X_train_s1.shape[1])

In [7]:
#danach trainieren und ergebniss auswerfen
clf = RocketClassifier(num_kernels=300) 
clf.fit(X_train, np.array(y_train1)) 

RocketClassifier(num_kernels=300)

In [12]:
#Die daten die wir predicten wollen müssen auch noch preprocessed werden
df_test1.drop(["timestamp","DeviationID ValueY","CuStepNo ValueY"],axis=1,inplace=True)

C:\Users\paulh\AppData\Local\Temp\ipykernel_8516\3567498108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test1.drop(["timestamp","DeviationID ValueY","CuStepNo ValueY"],axis=1,inplace=True)


In [15]:
np.array(df_test1).reshape(1,39,df_test1.shape[0])

array([[[1.56720276e+02, 1.00000000e+02, 9.54861641e-02, ...,
         0.00000000e+00, 2.51028633e+01, 3.71093750e+01],
        [5.41268311e+01, 0.00000000e+00, 0.00000000e+00, ...,
         1.53797546e+02, 1.00000000e+02, 8.21759701e-02],
        [1.96238422e+00, 8.49550858e+01, 8.47846527e+01, ...,
         1.62312561e+02, 0.00000000e+00, 0.00000000e+00],
        ...,
        [7.34051270e+02, 0.00000000e+00, 0.00000000e+00, ...,
         1.50000000e+01, 2.29650040e+01, 2.10708082e-01],
        [2.00000003e-01, 0.00000000e+00, 1.69744040e-03, ...,
         7.24800781e+02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 5.31051636e+01, 3.25303833e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.72579661e-03]]])

In [16]:
clf.predict(np.array(df_test1).reshape(1,39,df_test1.shape[0]))

array([7], dtype=int64)